In [ ]:
%reload_ext autoreload
#%load_ext autoreload
%autoreload 1
%matplotlib inline

import datetime
import pandas as pd
#import modin.pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import umap
from tqdm import tqdm


#from sklearnex import patch_sklearn
#patch_sklearn()

from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

import os
import sys
import gc
import joblib

os.chdir("T:\\laupodteam\\AIOS\\Bram\\notebooks\\code_dev\\celldyn_embedder\\sandbox")
#os.chdir("/media/UMCU/notebooks/code_dev/celldyn_embedder/sandbox")
sns.set_style("whitegrid")

write_to_disk = False
read_from_disk = True

from numba import jit, njit, float32
from functools import lru_cache, cached_property
from numpy import linalg as la
from collections import defaultdict
#import ray
#ray.init()

from hembedder.utils.distance import poincarre_dist, fractional_distance, hyperboloid_dist, Distance

In [ ]:
#@njit(float32(float32[:],float32[:]), fastmath=True)
#def poincarre_dist(x,y):
#    return np.arccosh(\
#    1 + 2*(\
#        la.norm(x-y,ord = 2)**2/((1-la.norm(x,ord = 2)**2)*(1-la.norm(y,ord = 2)**2))
#        )
#    )
#@njit(float32(float32[:],float32[:], float32), fastmath=True)
#def fractional_distance(x, y, f=0.5):
#    return np.power(np.abs(np.sum(np.power(x-y, f))), 1/f)

In [ ]:
#cpath = "T:\laupodteam\AIOS\Bram\data\CellDyn\celldyn_cleaned_transformed_imputed_ALL_with_ratios.feather"
#cpath = "T:\laupodteam\AIOS\Bram\data\CellDyn\celldyn_FULL_transformed_df.feather"
cpath = "T:\laupodteam\AIOS\Bram\data\CellDyn\celldyn_checked.feather"
celldyn = pd.read_feather(cpath)
celldyn.columns = celldyn.columns.str.lower()


In [ ]:
sns.displot(data=celldyn, x='age', hue='gender', kde=True)

In [ ]:
sns.displot(data=celldyn, x='draw_hour', hue='gender', kde=True)

In [ ]:
sns.lineplot(data=celldyn.loc[celldyn.gender.isin(['M', 'F'])], x='draw_hour', y='age', hue='gender')

In [ ]:
# We want to extract weights based on the gender, the draw hour and the age such that the weighted mean of the age is equal to the day-average of the age for that gender
# w = 1+|a_t-a_d|/a_d
celldyn = celldyn.assign(weight_vector=np.minimum(celldyn.age/celldyn.age.mean(), celldyn.age.mean()/celldyn.age))
weighted_mean = lambda x: np.average(x, weights=celldyn.loc[x.index, 'weight_vector'].values)
age_plot = celldyn.groupby(['draw_hour', 'gender']).agg(age_weighted=('age', weighted_mean)).reset_index()

In [ ]:
sns.lineplot(data=age_plot[age_plot.gender.isin(['M', 'F'])], x='draw_hour', y='age_weighted', hue='gender')

### Patients who have measurements in the evening and during the day

In [ ]:
meas_per_patient = celldyn.shape[0]/celldyn.studyid_alle_celldyn.nunique()
print(f"There are on average {meas_per_patient} measurements per patient")

In [ ]:
counts_per_patient = pd.DataFrame(celldyn.groupby('studyid_alle_celldyn').size(),  
                        columns=['counts_per_patient'])
celldyn = celldyn.merge(counts_per_patient.reset_index(), how='left', on='studyid_alle_celldyn')

In [ ]:
meta_cols = ['asp_dt', 'analyzer', 'studyid_alle_celldyn', 
             'time_to_measurement', 'gender', 'age' , 'afname_dt',
             'day_of_year', 'week_of_year', 'year', 'first_day',
              'draw_hour', 'meas_hour', 'draw_minute', 'draw_hour_dec']

In [ ]:
fig, ax = plt.subplots(figsize=(18,7))
sns.violinplot(data=celldyn[celldyn.counts_per_patient<30], x='counts_per_patient', y='draw_hour')
plt.axhline(7)
plt.axhline(17)

There are three blocks: 
* routine block: from 7AM to 5PM, mostly poli perhaps?
* evening block: from 5PM to 12PM
* morning block: from 12PM to 7AM

In [ ]:
celldyn.loc[:,'care_block'] = celldyn.apply(lambda x: 'routine_block'\
                                    if (x['draw_hour']>=7) & (x['draw_hour']<17)
                                    else 'evening_block' if (x['draw_hour']>=17) & (x['draw_hour']<=23)
                                    else 'morning_block', axis=1
)
meta_cols.append('care_block')
celldyn = celldyn.assign(draw_minute=celldyn.afname_dt.dt.minute)

In [ ]:
celldyn = celldyn.assign(draw_hour_dec=celldyn[['draw_hour', 'draw_minute']].apply(lambda x: round(x[0]+x[1]/60, 1), axis=1))
meas_cols =[c for c in celldyn.columns if 'c_b_' in c or ':' in c]

In [ ]:
# function to set all values to their 1st and 99th percentile
def clip_outliers(df, cols, qL=0.01, qH=0.99):
    for col in cols:
        df.loc[:,col] = df[col].clip(df[col].quantile(qL), df[col].quantile(qH))
    return df

In [ ]:
plot_df = celldyn.sample(250000)
plot_df = plot_df.groupby('draw_hour_dec').c_b_bas.quantile([0.25, 0.5, 0.75]).to_frame().join(
          plot_df.groupby('draw_hour_dec').c_b_eos.quantile([0.25, 0.5, 0.75]).to_frame()).join(
          plot_df.groupby('draw_hour_dec').c_b_neu.quantile([0.25, 0.5, 0.75]).to_frame()).join(
          plot_df.groupby('draw_hour_dec').c_b_seg.quantile([0.25, 0.5, 0.75]).to_frame()).reset_index()

plot_df.rename(columns={'level_1': 'percentile'}, inplace=True)

fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(22,9))

sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_bas', color='black', ax=ax[0,0])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_eos', color='black', ax=ax[0,1])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_neu', color='black', ax=ax[1,0])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_seg', color='black', ax=ax[1,1])

ax[0,0].set_xlabel('Hour of the day')
ax[0,1].set_xlabel('Hour of the day')
ax[1,0].set_xlabel('Hour of the day')
ax[1,1].set_xlabel('Hour of the day')

plt.suptitle('Hourly variation in granulocytes', fontsize=16)

In [ ]:
plot_df = celldyn.sample(250000)
plot_df = plot_df.groupby('draw_hour_dec').c_b_mon.quantile([0.25, 0.5, 0.75]).to_frame().join(
          plot_df.groupby('draw_hour_dec').c_b_mone.quantile([0.25, 0.5, 0.75]).to_frame()).join(
          plot_df.groupby('draw_hour_dec').c_b_lym.quantile([0.25, 0.5, 0.75]).to_frame()).reset_index()
plot_df.rename(columns={'level_1': 'percentile'}, inplace=True)

fig, ax = plt.subplots(ncols=3, figsize=(22,4))
sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_mon', color='black', ax=ax[0])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_mone', color='black', ax=ax[1])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_lym', color='black', ax=ax[2])
# seg
ax[0].set_xlabel('Hour of the day')
ax[1].set_xlabel('Hour of the day')
ax[2].set_xlabel('Hour of the day')

plt.suptitle('Hourly variation in agranulocytes', fontsize=16)

In [ ]:
plot_df = celldyn.sample(250000)
plot_df = plot_df.groupby('draw_hour_dec').c_b_retc.quantile([0.25, 0.5, 0.75]).to_frame().join(
          plot_df.groupby('draw_hour_dec').c_b_rbci.quantile([0.25, 0.5, 0.75]).to_frame()).join(
          plot_df.groupby('draw_hour_dec').c_b_irf.quantile([0.25, 0.5, 0.75]).to_frame()).join(
          plot_df.groupby('draw_hour_dec').c_b_wbc.quantile([0.25, 0.5, 0.75]).to_frame()).reset_index()

plot_df.rename(columns={'level_1': 'percentile'}, inplace=True)
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(22,9))

sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_retc', color='black', ax=ax[0,0])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_rbci', color='black', ax=ax[0,1])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_irf', color='black', ax=ax[1,0])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_wbc', color='black', ax=ax[1,1])

ax[0,0].set_xlabel('Hour of the day')
ax[0,1].set_xlabel('Hour of the day')
ax[1,0].set_xlabel('Hour of the day')
ax[1,1].set_xlabel('Hour of the day')

ax[0,0].set_ylabel('Reticulocytes')
ax[0,1].set_ylabel('RBC impedance')
ax[1,0].set_ylabel('Immature reticuloctye fraction')
ax[1,1].set_ylabel('White blood cell count')

plt.suptitle('Hourly variation in red/white blood cells ', fontsize=16)

In [ ]:
plot_df = celldyn.sample(250000)
plot_df = plot_df.groupby('draw_hour_dec').c_b_plti.quantile([0.25, 0.5, 0.75]).to_frame().join(
          plot_df.groupby('draw_hour_dec').c_b_plto.quantile([0.25, 0.5, 0.75]).to_frame()).reset_index()

fig, ax = plt.subplots(ncols=2, figsize=(22,4))

sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_plti', color='black', ax=ax[0])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_plto', color='black', ax=ax[1])

ax[0].set_xlabel('Hour of the day')
ax[1].set_xlabel('Hour of the day')

ax[0].set_ylabel('PLT impedance')
ax[1].set_ylabel('PLT optics')

plt.suptitle('Hourly variation in platelets ', fontsize=16)

In [ ]:
plot_df = celldyn.sample(250000)
plot_df = plot_df.groupby('draw_hour_dec').c_b_mcv.quantile([0.25, 0.5, 0.75]).to_frame().join(
          plot_df.groupby('draw_hour_dec').c_b_mch.quantile([0.25, 0.5, 0.75]).to_frame()).join(
          plot_df.groupby('draw_hour_dec').c_b_mchc.quantile([0.25, 0.5, 0.75]).to_frame()).join(
          plot_df.groupby('draw_hour_dec').c_b_rdw.quantile([0.25, 0.5, 0.75]).to_frame()).reset_index()
          
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(22,9))

sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_mcv', color='black', ax=ax[0,0])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_mch', color='black', ax=ax[0,1])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_mchc', color='black', ax=ax[1,0])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_rdw', color='black', ax=ax[1,1])


ax[0,0].set_xlabel('Hour of the day')
ax[0,1].set_xlabel('Hour of the day')
ax[1,0].set_xlabel('Hour of the day')
ax[1,1].set_xlabel('Hour of the day')

ax[0,0].set_ylabel('Mean corpuscular volume')
ax[0,1].set_ylabel('Mean corpuscular hemoglobin')
ax[1,0].set_ylabel('Mean corpuscular hemoglobin concentration')
ax[1,1].set_ylabel('Red cell distribution width')

plt.suptitle('Hourly variation of aggregate blood parameters ', fontsize=16)

In [ ]:
plot_df = celldyn.sample(250000)
plot_df = plot_df.groupby('draw_hour_dec').c_b_plti.quantile([0.25, 0.5, 0.75]).to_frame().join(
          plot_df.groupby('draw_hour_dec').c_b_plto.quantile([0.25, 0.5, 0.75]).to_frame()).reset_index()

fig, ax = plt.subplots(ncols=2, figsize=(22,4))

sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_plti', color='black', ax=ax[0])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='c_b_plto', color='black', ax=ax[1])

ax[0].set_xlabel('Hour of the day')
ax[1].set_xlabel('Hour of the day')

ax[0].set_ylabel('PLT impedance')
ax[1].set_ylabel('PLT optics')

plt.suptitle('Hourly variation in platelets ', fontsize=16)

In [ ]:
# wbc/(retc+irf), neu/seg, lym/mon, wbc/plti, plti/(retc+irf), wbc/(mon+lym), ig/(bas+neu+eos), retc/hb, wbc/rbci, rbci/(retc+irf)

celldyn = celldyn.assign(wbc_over_rbc=0.5*celldyn.c_b_wbc/(celldyn.c_b_rbci+celldyn.c_b_rbco))
celldyn = celldyn.assign(neu_over_seg=celldyn.c_b_neu/celldyn.c_b_seg)
celldyn = celldyn.assign(neu_over_wbc=celldyn.c_b_neu/(celldyn.c_b_wbc-celldyn.c_b_neu))
celldyn = celldyn.assign(lym_over_mon=celldyn.c_b_lym/celldyn.c_b_mon)
celldyn = celldyn.assign(wbc_over_plt=celldyn.c_b_wbc/celldyn.plt) # plti, plto
celldyn = celldyn.assign(plt_over_rbc=0.5*celldyn.plt/(celldyn.c_b_rbci+celldyn.c_b_rbco))
celldyn = celldyn.assign(retc_over_hb=celldyn.c_b_retc/celldyn.c_b_hb)
celldyn = celldyn.assign(rbc_over_irbc=celldyn.c_b_rbci/(celldyn.c_b_retc+celldyn.c_b_irf))
celldyn = celldyn.assign(plt_over_lym=celldyn.plt/celldyn.c_b_lym)

In [ ]:
# wbc_over_rbc
# neu_over_seg
# plt_over_rbc
# wbc_over_plt

plot_df = celldyn.sample(250000)
plot_df = plot_df.groupby('draw_hour_dec').wbc_over_rbc.quantile([0.25, 0.5, 0.75]).to_frame().join(
          plot_df.groupby('draw_hour_dec').neu_over_seg.quantile([0.25, 0.5, 0.75]).to_frame()).join(
          plot_df.groupby('draw_hour_dec').plt_over_rbc.quantile([0.25, 0.5, 0.75]).to_frame()).join(
          plot_df.groupby('draw_hour_dec').wbc_over_plt.quantile([0.25, 0.5, 0.75]).to_frame()).reset_index()
          
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(22,9))

sns.lineplot(data=plot_df, x='draw_hour_dec', y='wbc_over_rbc', color='black', ax=ax[0,0])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='neu_over_seg', color='black', ax=ax[0,1])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='plt_over_rbc', color='black', ax=ax[1,0])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='wbc_over_plt', color='black', ax=ax[1,1])


ax[0,0].set_xlabel('Hour of the day')
ax[0,1].set_xlabel('Hour of the day')
ax[1,0].set_xlabel('Hour of the day')
ax[1,1].set_xlabel('Hour of the day')

ax[0,0].set_ylabel('Leukocyte/erytrocyte ratio')
ax[0,1].set_ylabel('Neutrophil/segment ratio')
ax[1,0].set_ylabel('Platelet/erytrocyte ratio')
ax[1,1].set_ylabel('Leukocyte/platelet ratio')

plt.suptitle('Hourly variation of hematology ratios ', fontsize=16)

In [ ]:
# 'neu:lym'
# 'mon:lym'
#  plt_over_lym
#  retc_over_hb

plot_df = celldyn.sample(250000)
plot_df = plot_df.groupby('draw_hour_dec')['neu:lym'].quantile([0.25, 0.5, 0.75]).to_frame().join(
          plot_df.groupby('draw_hour_dec')['mon:lym'].quantile([0.25, 0.5, 0.75]).to_frame()).join(
          plot_df.groupby('draw_hour_dec').plt_over_lym.quantile([0.25, 0.5, 0.75]).to_frame()).join(
          plot_df.groupby('draw_hour_dec').retc_over_hb.quantile([0.25, 0.5, 0.75]).to_frame()).reset_index()
          
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(22,9))

sns.lineplot(data=plot_df, x='draw_hour_dec', y='neu:lym', color='black', ax=ax[0,0])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='mon:lym', color='black', ax=ax[0,1])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='plt_over_lym', color='black', ax=ax[1,0])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='retc_over_hb', color='black', ax=ax[1,1])


ax[0,0].set_xlabel('Hour of the day')
ax[0,1].set_xlabel('Hour of the day')
ax[1,0].set_xlabel('Hour of the day')
ax[1,1].set_xlabel('Hour of the day')

ax[0,0].set_ylabel('Neutrophil/lymphocyte ratio')
ax[0,1].set_ylabel('Monocyte/lymphocyte ratio')
ax[1,0].set_ylabel('Platelet/lymphocyte ratio')
ax[1,1].set_ylabel('Reticulocyte/hemoglobin ratio')

plt.suptitle('Hourly variation of hematology ratios ', fontsize=16)

In [ ]:
meas_cols.extend(['wbc_over_rbc', 'neu_over_seg', 'lym_over_mon', 'wbc_over_plt', 'neu_over_wbc',
                  'plt_over_rbc', 'retc_over_hb', 'rbc_over_irbc', 'plt_over_lym'])

# Identify multi-collinear compounds

In [ ]:
feature_correlations = []
for i in tqdm(range(20)):
    feature_correlations.append(celldyn.sample(5000)[meas_cols].corr(method='kendall'))

In [ ]:
from sklearn.cluster import AffinityPropagation

In [ ]:
w = 3
af_clust = AffinityPropagation(random_state=0, affinity='precomputed')
af_clust.fit(1-np.exp(w*(1-feature_correlations[0])))

In [ ]:
count_dict = defaultdict(int)
for corss in feature_correlations:
    for exemplar in corss.columns[af_clust.cluster_centers_indices_]:
        count_dict[exemplar] += 1/len(feature_correlations)

In [155]:
pd.DataFrame(count_dict, index=[1]).T.rename(columns={1:'count'}).sort_values('count', ascending=False)[:10]

,count
c_b_phpr,1.00
c_b_mchc,0.98
c_b_pig,0.97
c_b_limn,0.96
c_b_plto,0.96
c_b_pdw,0.94
c_b_irf,0.90
c_b_pmac,0.86
c_b_rbcfmn,0.84
c_b_pimn,0.82


In [ ]:
# use hierarchical clustering to find highly correlated feature-clusters


# choose examplars, put the other conntributing features in a dict with the examplar as key

## Reduce dimensionality

In [ ]:
from umap import UMAP
from sklearn.decomposition import PCA, FactorAnalysis, NMF, SparseCoder, SparsePCA
from sklearn.preprocessing import StandardScaler, QuantileTransformer, FunctionTransformer, MinMaxScaler
from sklearn.pipeline import Pipeline

In [ ]:
reduced_dimensions = 6
# manhattan: 40s, 50.000 samples
# poincarre: 4m, 50.000 samples
# fractional_distance: 2m, 50.000 samples
dist =  fractional_distance #, poincarre_dist, manhattan, hyperboloid_dist
reducer = UMAP(n_components=reduced_dimensions, n_neighbors=50, n_jobs=8, min_dist=0, metric=dist, densmap=False)
emb_cols = [f'dim_{i}' for i in range(reduced_dimensions)]

sample_index = celldyn.dropna(subset=meas_cols).sample(250000).index
celldyn_embedded = celldyn.loc[sample_index, meta_cols].copy()
reduce_pipe = Pipeline([
    #('Drop nan', FunctionTransformer(lambda x: x.dropna(axis=0, how='any'))),
    ('Scaling Q', QuantileTransformer(n_quantiles=500, output_distribution='normal', random_state=42)),
    ('Embedder', reducer),
])
reduce_pipe.fit(celldyn.loc[sample_index, meas_cols])
celldyn_embedded.loc[:, emb_cols] = reduce_pipe.transform(celldyn.loc[sample_index, meas_cols])
celldyn_embedded = celldyn_embedded.assign(normal_prick=celldyn_embedded.draw_hour==8)

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter_3d(celldyn_embedded.sample(10000), x='dim_0', y='dim_2', z='dim_4',
                    color='age', size_max=7, opacity=0.15)

# tight layout
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

In [ ]:
plot_df = celldyn_embedded.sample(250000)
plot_df = plot_df.groupby('draw_hour_dec').dim_0.quantile([0.25, 0.5, 0.75]).to_frame().join(
          plot_df.groupby('draw_hour_dec').dim_1.quantile([0.25, 0.5, 0.75]).to_frame()).join(
          plot_df.groupby('draw_hour_dec').dim_2.quantile([0.25, 0.5, 0.75]).to_frame()).join(
          plot_df.groupby('draw_hour_dec').dim_3.quantile([0.25, 0.5, 0.75]).to_frame()).join(
          plot_df.groupby('draw_hour_dec').dim_4.quantile([0.25, 0.5, 0.75]).to_frame()).join(
          plot_df.groupby('draw_hour_dec').dim_5.quantile([0.25, 0.5, 0.75]).to_frame()).reset_index()

          
fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(22,12))

sns.lineplot(data=plot_df, x='draw_hour_dec', y='dim_0', color='black', ax=ax[0,0])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='dim_1', color='black', ax=ax[0,1])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='dim_2', color='black', ax=ax[0,2])

sns.lineplot(data=plot_df, x='draw_hour_dec', y='dim_3', color='black', ax=ax[1,0])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='dim_4', color='black', ax=ax[1,1])
sns.lineplot(data=plot_df, x='draw_hour_dec', y='dim_5', color='black', ax=ax[1,2])

ax[0,0].set_xlabel('Hour of the day')
ax[0,1].set_xlabel('Hour of the day')
ax[0,2].set_xlabel('Hour of the day')
ax[1,0].set_xlabel('Hour of the day')
ax[1,1].set_xlabel('Hour of the day')
ax[1,2].set_xlabel('Hour of the day')

ax[0,0].set_ylabel('embedded dimension 0')
ax[0,1].set_ylabel('embedded dimension 1')
ax[0,2].set_ylabel('embedded dimension 2')

ax[1,0].set_ylabel('embedded dimension 3')
ax[1,1].set_ylabel('embedded dimension 4')
ax[1,2].set_ylabel('embedded dimension 5')

plt.suptitle('Hourly variation of embedded values ', fontsize=16)

In [ ]:
plot_df = celldyn_embedded.loc[celldyn_embedded.age.between(0,90)]

fig, ax = plt.subplots(ncols=3, nrows=2, figsize=(22,12))

sns.lineplot(data=plot_df, x='age', y='dim_0', color='black', ax=ax[0,0])
sns.lineplot(data=plot_df, x='age', y='dim_1', color='black', ax=ax[0,1])
sns.lineplot(data=plot_df, x='age', y='dim_2', color='black', ax=ax[0,2])

sns.lineplot(data=plot_df, x='age', y='dim_3', color='black', ax=ax[1,0])
sns.lineplot(data=plot_df, x='age', y='dim_4', color='black', ax=ax[1,1])
sns.lineplot(data=plot_df, x='age', y='dim_5', color='black', ax=ax[1,2])

ax[0,0].set_xlabel('Age')
ax[0,1].set_xlabel('Age')
ax[0,2].set_xlabel('Age')
ax[1,0].set_xlabel('Age')
ax[1,1].set_xlabel('Age')
ax[1,2].set_xlabel('Age')

ax[0,0].set_ylabel('embedded dimension 0')
ax[0,1].set_ylabel('embedded dimension 1')
ax[0,2].set_ylabel('embedded dimension 2')

ax[1,0].set_ylabel('embedded dimension 3')
ax[1,1].set_ylabel('embedded dimension 4')
ax[1,2].set_ylabel('embedded dimension 5')

plt.suptitle('Age variation of embedded values ', fontsize=16)

In [ ]:
PCA_reducer = PCA()
reduce_pipe = Pipeline([
    #('Drop nan', FunctionTransformer(lambda x: x.dropna(axis=0, how='any'))),
    ('Scaling Q', QuantileTransformer(n_quantiles=500, output_distribution='normal', random_state=42)),
    ('Reduce', PCA_reducer)])
reduce_pipe.fit(celldyn.loc[sample_index, meas_cols])

In [ ]:
plt.plot(np.cumsum(reduce_pipe.named_steps['Reduce'].explained_variance_ratio_))
plt.axhline(0.95, color='red')

In [ ]:
n_comps = 3
PCA_reducer = FactorAnalysis(rotation='varimax', n_components=n_comps)
reduce_pipe = Pipeline([
    #('Drop nan', FunctionTransformer(lambda x: x.dropna(axis=0, how='any'))),
    ('Scaling Q', QuantileTransformer(n_quantiles=500, output_distribution='normal', random_state=42)),
    ('Reduce', PCA_reducer)])
reduce_pipe.fit(celldyn.loc[sample_index, meas_cols])

FA_weights = pd.DataFrame(data=reduce_pipe.named_steps['Reduce'].components_, columns=meas_cols, index=['comp_{}'.format(i) for i in range(n_comps)])

In [148]:
FA_weights

,c_b_wbc,c_b_wvf,c_b_neu,c_b_seg,c_b_bnd,c_b_ig,c_b_lym,c_b_lyme,c_b_vlym,c_b_mon,...,neu:nrbc,wbc_over_rbc,neu_over_seg,lym_over_mon,wbc_over_plt,neu_over_wbc,plt_over_rbc,retc_over_hb,rbc_over_irbc,plt_over_lym
comp_0,0.215799,-0.205791,0.238170,0.148213,1.598315,1.694898,-0.124289,-0.131396,0.060746,0.112546,...,0.120689,0.244651,2.150929,-0.188384,0.267367,0.217237,-0.075389,0.052099,-0.054408,-0.011369
comp_1,0.012981,0.038191,-0.037168,-0.033661,-0.097529,-0.077577,0.185599,0.193665,-0.034965,0.061773,...,-0.040311,-0.017763,-0.101643,0.097947,-0.112147,-0.159307,0.103847,-0.024668,0.028633,-0.047375
comp_2,-0.378719,-0.293683,-0.680559,-0.694372,-0.156038,-0.023206,0.613455,0.608562,0.513392,-0.123026,...,-0.630497,-0.424976,-0.021709,0.656281,-0.262412,-0.924957,-0.140574,-0.164959,0.159299,-0.603898


In [ ]:
NMF_reducer = NMF(l1_ratio=1, max_iter=2000, n_components=32)
reduce_pipe = Pipeline([
    #('Drop nan', FunctionTransformer(lambda x: x.dropna(axis=0, how='any'))),
    ('Scaling Q', QuantileTransformer(n_quantiles=500, output_distribution='uniform', random_state=42)),
    ('Reduce', NMF_reducer)])
reduce_pipe.fit(celldyn.loc[sample_index, meas_cols])

NMF_weights = pd.DataFrame(data=reduce_pipe.named_steps['Reduce'].components_, columns=meas_cols, index=['comp_{}'.format(i) for i in range(len(meas_cols))])

In [ ]:
NMF_reducer.reconstruction_err_

In [ ]:
NMF_weights.sum(axis=0).sort_values(ascending=False)[:25]

In [ ]:
sPCA_reducer = SparsePCA(method='lars', max_iter=1500)
reduce_pipe = Pipeline([
    #('Drop nan', FunctionTransformer(lambda x: x.dropna(axis=0, how='any'))),
    #('Scaling Q', QuantileTransformer(n_quantiles=100, output_distribution='normal', random_state=123)),
    ('Scaling S', StandardScaler()),
    ('Reduce', sPCA_reducer)])
reduce_pipe.fit(celldyn.loc[sample_index, meas_cols])

sPCA_weights = pd.DataFrame(data=reduce_pipe.named_steps['Reduce'].components_, columns=meas_cols, index=['comp_{}'.format(i) for i in range(len(meas_cols))])

In [ ]:
sPCA_weights.columns[(sPCA_weights!=0).sum(axis=0)>1]

In [ ]:
qt =  QuantileTransformer(n_quantiles=50, output_distribution='normal', random_state=123)
sparse_coded_celldyn = SparseCoder(dictionary=qt.fit_transform(celldyn.loc[sample_index, meas_cols].T.values))\
                                                .fit_transform(celldyn.loc[sample_index, meas_cols].T.values)
sCoded_recon = pd.DataFrame(data=sparse_coded_celldyn, columns=meas_cols, index=['comp_{}'.format(i) for i in range(len(meas_cols))])

In [ ]:
from scipy.spatial.distance import pdist
from scipy.spatial.distance import cdist
from scipy.spatial import distance

from sklearn.metrics import cohen_kappa_score, matthews_corrcoef

import dcor

In [ ]:
# how important is each component in reconstructing the original data
# - sub sample
#   -- determine the distance matrix between the samples on the basis of the original components
#   -- randomly permute the components and retry

def get_distance(X: np.array)-> np.array:
    return pdist(X, metric='cityblock')

def get_rank(D)-> np.array:
    return np.argsort(D)

@lru_cache(maxsize=None)
def permute_vector(l=1000, n=0):
    return np.random.permutation(l)

def get_permuted_rank(X, n_perm=10):
    num_features = X.shape[1]
    num_samples = X.shape[0]

    perm_vectors = [permute_vector(l=num_samples, n=n) for n in range(n_perm)]

    res_array = []    
    for d in range(num_features):
        sub_list = []
        for n in range(n_perm):
            X_perm = X.copy()
            X_perm[:, d] = X[perm_vectors[n], d]
            rank_vector = get_rank(get_distance(X_perm))
            sub_list.append(rank_vector)
        res_array.append(sub_list)
    return res_array

def get_perturbed_rank(X, n_pert=10):
    num_features = X.shape[1]
    num_samples = X.shape[0]

    stats_list = []
    for d in range(num_features):
        stats_list.append((np.mean(X[:, d]), np.std(X[:, d])))

    res_array = []    
    for d in range(num_features):
        sub_list = []
        for n in range(n_pert):
            perburbation_vector = np.random.normal(loc=0, scale=2*stats_list[d][1], size=num_samples)
            X_perb = X.copy()
            X_perb[:, d] = X[:, d] + perburbation_vector
            rank_vector = get_rank(get_distance(X_perb))
            sub_list.append(rank_vector)
        res_array.append(sub_list)
    return res_array

def rank_match(v1,v2):
    return np.sum(v1-v2)/len(v1)

def repeated_agreement_metric(v1, v2, num_rounds=100, num_per_round=100):
    scores = []
    for _ in range(num_rounds):
        batch = np.random.choice(len(v1), size=num_per_round, replace=True)
        score = cohen_kappa_score(v1[batch], v2[batch])
        scores.append(score)
    return np.mean(scores)

def compare_rank_with_original(X, rank_metric='correlation', n_perm=10, n_pert=10, how='permutation'):
    orank = get_rank(get_distance(X))

    if how == 'permutation':
        changed_ranks = get_permuted_rank(X, n_perm=n_perm)
    elif how== 'perturbation':
        changed_ranks = get_perturbed_rank(X, n_pert=n_pert)
    else:
        raise ValueError('how should be either permutation or perturbation')

    # rank-distance
    res = []
    for d in range(X.shape[1]):
        sub_res = []
        for prank in changed_ranks[d]:
            sub_res.append(repeated_agreement_metric(orank.ravel(), prank.ravel()))
        res.append(sub_res)
    return res

def get_permuted_distance_correlation(X, n_perm=10):
    num_features = X.shape[1]
    num_samples = X.shape[0]
    res = []
    for d in tqdm(range(num_features)):
        sub_list = []
        for n in range(n_perm):
            X_perm = X.copy()
            X_perm[:, d] = X[permute_vector(l=num_samples, n=n), d]
            sub_list.append(dcor.distance_correlation(X, X_perm))
        res.append(sub_list)
    return res

def get_perturbed_distance_correlation(X, n_perm=10):
    num_features = X.shape[1]
    num_samples = X.shape[0]
        	
    stats_list = []
    for d in range(num_features):
        stats_list.append((np.mean(X[:, d]), np.std(X[:, d])))

    res = []
    for d in tqdm(range(num_features)):
        sub_list = []
        for n in range(n_perm):
            perburbation_vector = np.random.normal(loc=1, scale=4*stats_list[d][1]/np.abs(stats_list[d][0]), size=num_samples)
            X_perm = X.copy()
            X_perm[:, d] = X[:, d] * perburbation_vector
            dist = dcor.distance_correlation(X, X_perm)
            sub_list.append(dist)
        res.append(sub_list)
    return res


In [ ]:
np_perm = 5
np_pert = 5
sample_size = 1000
rank_results = compare_rank_with_original(QuantileTransformer(output_distribution='normal', n_quantiles=100)\
                            .fit_transform(celldyn.dropna().sample(sample_size).loc[:, meas_cols].values), n_perm=np_perm, n_pert=np_pert, how='perturbation')
rank_results_df = pd.DataFrame(rank_results, index=meas_cols, columns=[f'round_{i}' for i in range(np_perm)])

In [ ]:
np_perm = 100
dist_results = get_perturbed_distance_correlation(QuantileTransformer(output_distribution='normal', n_quantiles=100).fit_transform(celldyn.dropna().sample(sample_size).loc[:, meas_cols].values), n_perm=np_perm)
dist_results_df = pd.DataFrame(dist_results, index=meas_cols, columns=[f'round_{i}' for i in range(np_perm)])

In [ ]:
rank_results_df.mean(axis=1).sort_values(ascending=False)[:20]

In [ ]:
dist_results_df.mean(axis=1).sort_values(ascending=False)[:20]

# Concrete AutoEncoder

In [ ]:
from concrete_autoencoder import ConcreteAutoencoderFeatureSelector
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, LeakyReLU
from sklearn.model_selection import train_test_split

In [ ]:
X  = QuantileTransformer(output_distribution='normal', n_quantiles=250).fit_transform(celldyn.dropna().sample(100000).loc[:, meas_cols].values)

num_rounds = 100
feature_selection = []
for _ in tqdm(range(num_rounds)):
    x_train, x_test, _, _ = train_test_split(X, X, test_size=0.2, random_state=None)
    x_train = np.reshape(x_train, (len(x_train), -1))
    x_test = np.reshape(x_test, (len(x_test), -1))

    def decoder(x):
        x = Dense(512)(x)
        x = LeakyReLU(0.2)(x)
        x = Dropout(0.1)(x)
        x = Dense(512)(x)
        x = LeakyReLU(0.2)(x)
        x = Dropout(0.1)(x)
        x = Dense(X.shape[1])(x)
        return x

    selector = ConcreteAutoencoderFeatureSelector(K = 32, output_function = decoder, num_epochs = 450)

    selector.fit(x_train, x_train, x_test, x_test)
    feature_selection.append(np.array(meas_cols)[selector.get_support(indices=True)])

In [ ]:
count_dict = defaultdict(int)

for i in feature_selection:
    for j in i:
        count_dict[j] += 1/len(feature_selection)
        
    presence_df = pd.DataFrame(count_dict, index=[0]).T.sort_values(by=0, ascending=False).rename(columns={0:'presence'})

In [158]:
presence_df[:20]

,presence
c_b_phpr,1.00
c_b_mchc,0.98
c_b_pig,0.97
c_b_limn,0.96
c_b_plto,0.96
c_b_pdw,0.94
c_b_irf,0.90
c_b_pmac,0.86
c_b_rbcfmn,0.84
c_b_pimn,0.82


In [ ]:
# https://github.com/bwohlberg/sporco